介绍了python中的偏函数（partial function）<br>

偏函数和装饰器一样，它可以扩展函数的功能，但又不完成等价于装饰器。<br>
通常应用的场景是当我们要频繁调用某个函数时，其中某些参数是已知的固定值，如果我们反复调用这个函数多次， 看上去似乎代码有些冗余，而偏函数的出现就是为了很少的解决这一个问题。

In [1]:
from functools import partial

In [13]:
"""
func = functools.partial(func, *args, **keywords)
func: 需要被扩展的函数，返回的函数其实是一个类 func 的函数
*args: 需要被固定的位置参数
**kwargs: 需要被固定的关键字参数
如果在原来的函数 func 中关键字不存在，将会扩展，如果存在，则会覆盖
"""
print("")

#### 示例1：多个数求和

In [5]:
# 求三个数（其中一个固定为100）的和
# 方法一：直接在函数中写入，但这样在100变动时需要直接修改原函数
def add_1(*args):
    return sum(args) + 100
print(add_1(20, 30))

150


In [7]:
# 方法二：在外部函数中写入，但这样的封装对于复杂功能显得很low
def add_2(*args):
    return sum(args)
print(add_2(20, 30)+100)

150


In [9]:
# 方法三：利用偏函数进行重构
def add_3(*args):
    return sum(args)

add = partial(add_3, 100)
print(add(20, 30))

150


In [10]:
add = partial(add_3, 200)
print(add(20, 30))

250


#### DIY的偏函数

In [39]:
# 偏函数相当于如下功能
def partial_DIY(func, *args, **kwargs):
    def newfunc(*fargs, **fkwargs):
        fixed_kwargs = kwargs.copy()
        fkwargs.update(fixed_kwargs)
        return func(*args, *fargs, **fkwargs)
    return newfunc

In [37]:
def add_4(*args):
        return sum(args)

add_diy = partial_DIY(add_4, 100)
print(add_diy(20, 30))

150


#### 示例2： 求多点对固定点的距离

当然，可以直接用Numpy进行广播操作，这里尝试用偏函数做

In [42]:
# 随机生成一组数
import random
data_generator = lambda: random.randint(0,10)
sources = [(data_generator(), data_generator()) for _ in range(10)]
sources

[(6, 6),
 (10, 5),
 (0, 8),
 (6, 6),
 (10, 0),
 (10, 8),
 (8, 3),
 (2, 3),
 (10, 1),
 (1, 1)]

In [43]:
# 距离计算
import math
def euclid_dist(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

In [44]:
# 结合固定的目标点坐标定义为偏函数
target = (5, 5)
partial_euclid_dist = partial(euclid_dist, target)
partial_euclid_dist((0, 0))

7.0710678118654755

In [45]:
sources.sort(key=partial_euclid_dist)

In [46]:
sources

[(6, 6),
 (6, 6),
 (8, 3),
 (2, 3),
 (10, 5),
 (1, 1),
 (0, 8),
 (10, 8),
 (10, 1),
 (10, 0)]

#### 示例3：if-else逻辑的统一管理

以一段文字的正则化处理为例，其涉及到冗长而复杂的逻辑判断，可以通过偏函数进行统一管理

##### 直接硬编码

In [59]:
text_raw = "12345678\nAcademic\nBonus"
text = text_raw.split('\n')
text

['12345678', 'Academic', 'Bonus']

In [60]:
import re
for t in text:
    if re.search(r'^A', t):
        print("文本以A开头")
    elif re.search(r'^B', t):
        print("文本以B开头") 
    elif re.search(r'^\d', t):
        print("文本以数字开头")

文本以数字开头
文本以A开头
文本以B开头


这种功能写法的问题在于未对正则匹配进行命名，可能会导致阅读的问题

##### 进行函数封装

In [63]:
def starts_with_A(t):
    return re.search(r'^A', t)

def starts_with_B(t):
    return re.search(r'^B', t)

def starts_with_digit(t):
    return re.search(r'^\d', t)

for t in text:
    if starts_with_A(t):
        print("文本以A开头")
    elif starts_with_B(t):
        print("文本以B开头")
    elif starts_with_digit(t):
        print("文本以数字开头")

文本以数字开头
文本以A开头
文本以B开头


##### 进一步进行偏函数的封装

In [64]:
starts_with_A_partial = partial(re.search, r'^A')
starts_with_B_partial = partial(re.search, r'^B')
starts_with_digit_partial = partial(re.search, r'^\d')
for t in text:
    if starts_with_A_partial(t):
        print("文本以A开头")
    elif starts_with_B_partial(t):
        print("文本以B开头")
    elif starts_with_digit_partial(t):
        print("文本以数字开头")

文本以数字开头
文本以A开头
文本以B开头
